# First Act: Establish Particles in Our System
The goal of this first section is to be able to specify the particles in our system, the independent and dependent variables, write the dependent variables in terms of the independent variables using Baryon num. conservation and charge conservation, and obtain the $\tilde{\mu}_i$. 

Then, we can write down the system of equations 

In [357]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sympy as sym
from scipy import optimize

sym.init_printing(use_latex='mathjax')

pi = np.pi
Pi = sym.symbols('pi')

## Declare Baryon and Lepton Classes
1. Declare symbolic classes 
2. Goal: write a function that takes names of all baryons (in our system) as input and declares a baryon object class for each one... and also re-writes the dependent variables in terms of independent variables. Basically does the entire set up... It might pay to simply write a dictionary pre-filled with everything.

In [274]:
Neutron = [sym.symbols('n_n'), sym.symbols('n_B')*sym.symbols('x_n'), sym.symbols('x_n'), sym.symbols('mu_n'), 0.0]
Proton = [sym.symbols('n_p'), sym.symbols('n_B')*sym.symbols('x_p'), sym.symbols('x_p'), sym.symbols('mu_p'), 1.0]
Lambda_0 = [sym.symbols('n_Lambda_0'), sym.symbols('n_B')*sym.symbols('x_Lambda_0'), sym.symbols('x_Lambda_0'),\
             sym.symbols('mu_Lambda_0'),  0.0]
Sigma_0 = [sym.symbols('n_Sigma_0'), sym.symbols('n_B')*sym.symbols('x_Sigma_0'), sym.symbols('x_Sigma_0'),\
           sym.symbols('mu_Sigma_0'), 0.0]
Sigma_neg = [sym.symbols('n_Sigma_-'), sym.symbols('n_B')*sym.symbols('x_Sigma_-'), sym.symbols('x_Sigma_-'),\
           sym.symbols('mu_Sigma_-'), 0.0]
Cascade_neg = [sym.symbols('n_\Xi_-'), sym.symbols('n_B')*sym.symbols('x_\Xi-'), sym.symbols('x_\Xi-'),\
               sym.symbols('mu_\Xi_-'), -1.0]

In [275]:
total_baryon_list = [Neutron, Proton, Lambda_0, Sigma_0, Sigma_neg, Cascade_neg]
total_baryon_list_name = ['Neutron', 'Proton', 'Lambda_0', 'Sigma_0', 'Sigma_neg', 'Cascade_neg']

In [276]:
A = pd.DataFrame(Neutron, index = ['Density', 'nB*Fraction' ,'Fraction', 'Chem Pot', 'Charge' ], columns = ['Neutron'])

In [277]:
for i in range(len(total_baryon_list)):
    A[total_baryon_list_name[i]] = total_baryon_list[i]

In [278]:
A

,Neutron,Proton,Lambda_0,Sigma_0,Sigma_neg,Cascade_neg
Density,n_n,n_p,n_Lambda_0,n_Sigma_0,n_Sigma_-,n_\Xi_-
nB*Fraction,n_B*x_n,n_B*x_p,n_B*x_Lambda_0,n_B*x_Sigma_0,n_B*x_Sigma_-,n_B*x_\Xi-
Fraction,x_n,x_p,x_Lambda_0,x_Sigma_0,x_Sigma_-,x_\Xi-
Chem Pot,mu_n,mu_p,mu_Lambda_0,mu_Sigma_0,mu_Sigma_-,mu_\Xi_-
Charge,0.0,1.0,0.0,0.0,0.0,-1.0


In [279]:
Electron = [sym.symbols('n_e'), sym.symbols('n_e')*sym.symbols('x_e'), sym.symbols('x_e'), sym.symbols('mu_e'), -1.0]
Muon = [sym.symbols('n_\mu'), sym.symbols('n_\mu')*sym.symbols('x_\mu'), sym.symbols('x_\mu'), sym.symbols('mu_mu'), -1.0]

In [280]:
total_lepton_list = [Electron, Muon]
total_lepton_list_name = ['Electron', 'Muon']

In [281]:
B = pd.DataFrame(Electron, index = ['Density', 'nB*Fraction' ,'Fraction', 'Chem Pot', 'Charge' ], columns = ['Electron'])
for i in range(len(total_lepton_list)):
    B[total_lepton_list_name[i]] = total_lepton_list[i]

In [348]:
class baryon_sym:
    def __init__(self, num_dens, num_dens_frac, frac, chem_pot, charge):
        self.num_dens = num_dens
        self.num_dens_frac = num_dens_frac
        self.frac = frac
        self.chem_pot = chem_pot
        self.charge = charge
        
class lepton_sym:
    def __init__(self, num_dens, num_dens_frac, frac, chem_pot, charge):
        self.num_dens = num_dens
        self.num_dens_frac = num_dens_frac
        self.frac = frac
        self.chem_pot = chem_pot
        self.charge = charge 
        
class independent_variables:
    def __init__(self, frac, tilde_chem_pot):
        self.frac = frac
        self.tilde_chem_pot = tilde_chem_pot
        
        # to be filled in later
        self.tilde_chem_pot_val = 0.0 

In [283]:
def baryon(baryon_string):
    return baryon_sym(A.loc['Density', baryon_string], A.loc['nB*Fraction', baryon_string],\
                             A.loc['Fraction', baryon_string], A.loc['Chem Pot', baryon_string]\
                              , A.loc['Charge', baryon_string])

def lepton(lepton_string):
    return lepton_sym(B.loc['Density', lepton_string], B.loc['nB*Fraction', lepton_string],\
                             B.loc['Fraction', lepton_string], B.loc['Chem Pot', lepton_string]\
                              ,B.loc['Charge', lepton_string])

In [372]:
neutron = baryon('Neutron')
proton = baryon('Proton')
lambda_0 = baryon('Lambda_0')
#sigma_0 = baryon('Sigma_0')
sigma_neg = baryon('Sigma_neg')
cascade_neg = baryon('Cascade_neg')

electron = lepton('Electron')
muon = lepton('Muon')

electron_frac = independent_variables(electron.frac, sym.symbols('mutilde_xe'))
muon_frac = independent_variables(muon.frac, sym.symbols('mutilde_x_mu'))
lambda_0_frac = independent_variables(lambda_0.frac, sym.symbols('mutilde_x_Lambda_0'))
sigma_neg_frac = independent_variables(sigma_neg.frac, sym.symbols('mutilde_x_Sigma_-'))
cascade_neg_frac = independent_variables(cascade_neg.frac, sym.symbols('mutilde_x_Xi_-'))


baryon_list = [neutron, proton, lambda_0, sigma_neg, cascade_neg]
lepton_list = [electron, muon]

independent_var_list = [electron_frac, muon_frac, lambda_0_frac, sigma_neg_frac, cascade_neg_frac] 

## Declare Symbols: Independent and Dependent Variables

In [364]:
def baryon_num_cons(baryon_list):
    # takes list of baryons and returns baryon number conservation expression
    right = 0
    for baryon in baryon_list:
        right = baryon.frac + right 
    return sym.Eq(1, right)

In [365]:
baryon_num_cons(baryon_list)

1 = x_Λ_0 + x_Σ_- + x_\Xi- + xₙ + xₚ

In [366]:
def charge_cons(lepton_list, baryon_list):
    # takes list of both leptons and baryons and returns expression for charge conservation
    # Note: assumes that both lepton and baryon objects have a charge attribute
    
    positive, negative = 0, 0
    for lepton in lepton_list:
        if (lepton.charge > 0):
            positive = positive + lepton.frac
        elif (lepton.charge < 0):
            negative = negative + lepton.frac
    
    for baryon in baryon_list:
        if (baryon.charge > 0):
            positive = positive + baryon.frac
        elif (baryon.charge < 0):
            negative = negative + baryon.frac
    
    return sym.Eq(positive, negative)

In [367]:
charge_cons(lepton_list, baryon_list)

xₚ = x_\Xi- + x_\mu + xₑ

### Now with both Baryon number conservation and Charge conservation, we have two equations and two unknowns. Want to solve for unknowns. Just call a linsolve thingy

In [374]:
SOLVE = sym.linsolve([baryon_num_cons(baryon_list), charge_cons(lepton_list, baryon_list)], (proton.frac, neutron.frac))
SOLVE

{(x_\Xi- + x_\mu + xₑ, -x_Λ_0 - x_Σ_- - 2⋅x_\Xi- - x_\mu - xₑ + 1)}

### Then we update the proton and neutron frac in terms of the independent variables

In [375]:
proton.frac = SOLVE.args[0][0]
neutron.frac = SOLVE.args[0][1]
neutron.frac

-x_Λ_0 - x_Σ_- - 2⋅x_\Xi- - x_\mu - xₑ + 1

### Goal: Take independent variables and solve for dependent variables in terms of independent variables

Next step is to calculate
$$
    \mu_n - \mu_{\text{avg}} = \mu_n - \sum_i \mu_i x_i \qquad i \in \text{Baryons}
$$

In [361]:
def mu_tilde(lepton_list, baryon_list, independent_var_list):
    # returns mu tilde values
    
    # first construct mu_n - mu_average
    expression = 0 
    for baryon in baryon_list:
        expression = baryon.chem_pot*baryon.frac + expression
    for lepton in lepton_list:
        expression = lepton.chem_pot*lepton.frac + expression
    expression = neutron.chem_pot - expression
    display(expression)
    
    
    # differentiate to simplify things and store mu tilde values in independent variable object
    for var in independent_var_list:
        var.tilde_chem_pot_val = expression.diff(var.frac)

In [379]:
for var in independent_var_list:
    display(var.tilde_chem_pot_val)

-μₑ + μₙ - μₚ

-μₘᵤ + μₙ - μₚ

-μ_Λ_0 + μₙ

-μ_Σ_- + μₙ

-μ_\Xi_- + 2⋅μₙ - μₚ

## Getting the system of linear equations down